### 12.1 판다스와 모델코드의 인터페이스

In [1]:
import pandas as pd
data = pd.DataFrame({
    'x0':[1,2,3,4,5],
    'x1':[0.01,-0.01,0.25,-4.1,0.],
    'y':[-1.5,0.,3.6,1.3,-2.]
})

C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
df2 = pd.DataFrame(data.to_numpy(), columns=['one','two','three'])

In [6]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [7]:
df3 = data.copy()

In [8]:
df3['strings']=['a','b','c','d','e']

In [9]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [10]:
df3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [11]:
model_cols = ['x0','x1']
data.loc[:,model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [12]:
data['category'] = pd.Categorical(['a','b','a','a','b'], categories=['a','b'])

In [13]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [14]:
#만일 카테고리값을 더미값으로 치환하고 싶다면
#1. 더미값 생성 > 2. category 열 삭제
dummies = pd.get_dummies(data.category, prefix='category', dtype=float)
data_with_dummies = data.drop('category',axis=1).join(dummies)

In [15]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1.0,0.0
1,2,-0.01,0.0,0.0,1.0
2,3,0.25,3.6,1.0,0.0
3,4,-4.10,1.3,1.0,0.0
4,5,0.00,-2.0,0.0,1.0


### 12.2 patsy로 모델 생성하기
- 통계 모델을 위한 파이썬 라이브러리이며 R이나 S 통계 프로그래밍 언어에서 사용하는 **수식 문법**과 유사한 형식의 문자열 기반 수식 문법 제공
- 수식문법
  - a + b : a에 b를 더하는 것이 아니라 모델을 위해 생성된 배열을 설계

In [16]:
data = pd.DataFrame({
    'x0':[1,2,3,4,5],
    'x1':[0.01,-0.01,0.25,-4.1,0.],
    'y':[-1.5,0.,3.6,1.3,-2.]
})

In [17]:
import patsy

In [18]:
y,X = patsy.dmatrices('y ~x0 + x1', data) #DesignMatrix 인스턴스는 몇 가지 추가 메타데이터가 포함된 넘파이 ndarray
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [19]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [20]:
import numpy as np
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [21]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [22]:
#Intercept는 최소 제곱법같은 선형모델을 위한 표현
#모델에 0을 더해 절편 제거
patsy.dmatrices('y ~x0 + x1 + 0',data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [23]:
# 최소제곱법 회귀 분석을 위해 numpy.linalg.lstsq 같은 알고리듬에 바로 넘길 수도 있음
coef, resid, _,_ = np.linalg.lstsq(X, y, rcond=None)

In [24]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [25]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

#### 12.2.1 patsy 용법으로 데이터 변환하기

In [27]:
y, X = patsy.dmatrices('y ~ x0+np.log(np.abs(x1)+1)',data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [28]:
#자주 사용하는 변수 변환으로는 표준화와 중심화가 있는데 pasty에는 이런 목적을 위한 내장 함수 존재
y, X = patsy.dmatrices('y ~standardize(x0)+center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

- 모델링 과정에서 다른 모델을 기반으로 평가하는 경우가 생김
- 예를들어 홀드아웃이거나 신규 데이터가 나중에 관측되는 경우
  - **홀드아웃**: 일부 데이터를 의도적으로 사용하지 않음
- 따라서 새로운 데이터 기반으로 예측하는 용도로 모델을 사용 <br>
=> 상태를 가지는 변환

In [40]:
new_data = pd.DataFrame(data = pd.DataFrame({
    'x0':[6,7,8,9],
    'x1':[3.1,-0.5,0,2.3],
    'y':[1,2,3,4]
}))

In [39]:
#build_design_matrices함수는 입력 데이터셋에서 저장한 정보를 사용해 출력 데이터에 변환 적용
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [41]:
y,X = patsy.dmatrices('y ~ I(x0+x1)', data) #일반적인 +는 연산자가 아니므로 I로 둘러싼다
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

#### 12.2.2 범주형 데이터와 patsy
- patsy에서 숫자가 아닌 용법을 사용하면 기본적으로 더미변수로 변환
- 만약 절편이 존재하면 공선성을 피하기 위해 레벨 중 하나는 남겨둔다
  - 공선성 : 입력변수들 간의 상관정도가 높은 상태

In [43]:
data = pd.DataFrame({
    'key1':['a','a','b','b','a','b','a','b'],
    'key2':[0,1,0,1,0,1,0,0],
    'v1':[1,2,3,4,5,6,7,8],
    'v2':[-1,0,2.5,-0.5,4.0,-1.2,0.2, -1.7]
})

In [44]:
y,X = patsy.dmatrices('v2~key1',data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [45]:
#절편생략
y,X = patsy.dmatrices('v2~key1+0',data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [46]:
#숫자열은 C함수를 이용해서 범주형으로 해석
y,X = patsy.dmatrices('v2~C(key2)',data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [47]:
#여러 범주형 항을 사용하는 경우 분산분석 모델에서처럼 key1:key2 같은 용법 사용 가능
data['key2'] = data['key2'].map({0:'zero', 1:'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [48]:
y,X = patsy.dmatrices('v2~key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [49]:
y,X = patsy.dmatrices('v2~key1+key2+key1:key2',data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

### 12.3 statsmodels 소개
- 다양한 종류의 통계 모델 피팅, 통계 테스트 주행, 데이터 탐색, 시각화를위한 파이썬 라이브러리
- 선형모델, 일반 선형 모델, 로버스트 선형모델
- 선형 혼합 효과 모델
- 분산분석 메서드
- 시계열 처리 및 상태 공간 모델
- 일반적률추정법

#### 12.3.1 선형 모델 예측하기

In [50]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [51]:
rng = np.random.default_rng(seed = 12345)
def dnorm(mean, variance, size = 1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance)*rng.standard_normal(*size)

N=100
X = np.c_[dnorm(0, 0.4, size=N),
         dnorm(0, 0.6, size=N),
         dnorm(0,0.2, size=N)]
eps = dnorm(0,0.1,size=N) # 특정평균과 분산을 가지는 정규분포 데이터를 생성하는 헬퍼함수
beta = [0.1,0.3,0.5] #매개변수를 이용해서 진짜모델 작성

y=np.dot(X, beta)+eps

In [53]:
X[:5]

array([[-0.90050602, -0.18942958, -1.0278702 ],
       [ 0.79925205, -1.54598388, -0.32739708],
       [-0.55065483, -0.12025429,  0.32935899],
       [-0.16391555,  0.82403985,  0.20827485],
       [-0.04765129, -0.21314698, -0.04824364]])

In [54]:
y[:5]

array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445])

In [55]:
X_model = sm.add_constant(X) #기존 행렬에 절편열을 더한 함수

In [56]:
X_model[:5]

array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899],
       [ 1.        , -0.16391555,  0.82403985,  0.20827485],
       [ 1.        , -0.04765129, -0.21314698, -0.04824364]])

In [57]:
#최소제곱법 선형 회귀에 피팅
model = sm.OLS(y,X)

In [58]:
#fit메서드는 예측 모델 매개변수와 다른 분석 정보를 포함하는 회귀 결과 객체를 반환
results = model.fit()
results.params

array([0.06681503, 0.26803235, 0.45052319])

In [59]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Sat, 28 Oct 2023   Prob (F-statistic):                    2.66e-13
Time:                        14:23:26   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [60]:
#모든 모델 매개변수가 하나의 DataFrame에 들어있다고 가정해보자
data = pd.DataFrame(X, columns = ['col0','col1', 'col2'])

In [61]:
data['y'] = y

In [62]:
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [63]:
results = smf.ols('y~col0+col1+col2', data=data).fit()

In [64]:
results.params

Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [65]:
results.tvalues

Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64

In [66]:
#Series가 DataFrame의 열이름을 사용하고 있음을 알 수 있음
#판다스 객체를 이용해 수식을 사용하는 경우에는 add_constant를 호출할 필요가 없음
#샘플 데이터를 통해 예측 모델 매개변수에 전달한 예측값을 계산할 수 있음
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

#### 12.3.2 시계열 처리 예측하기
- 자기 회귀 구조와 노이즈를 이용해서 시계열 데이터를 실험

In [67]:
init_x = 4
values = [init_x, init_x]
N=1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0,0.1,N)
for i in range(N):
    new_x = values[-1]*b0+values[-2]*b1+noise[i]
    values.append(new_x)
    
#매개변수가 0.8, -0.4인 AR(2) 구조

In [69]:
from statsmodels.tsa.ar_model import AutoReg
MAXLAGS = 5
model = AutoReg(values, MAXLAGS)
results = model.fit()

In [70]:
results.params

array([ 0.02346612,  0.8096828 , -0.42865278, -0.03336517,  0.04267874,
       -0.05671529])

### 12.4 사이킷런 소개
- 가장 널리 사용되는 범요 파이썬 머신러닝 툴킷
- 사이킷런은 표준적인 지도&비지도 학습 메서드를 포함하고 있으며 모델 선택, 평가, 데이터변형, 적재, 모델유지 및 기타 작업을 위한 도구 제공
- 분류, 클러스터링, 예특, 여러 작업에 사용할 수 있음

In [71]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [72]:
#statsmodels나 사이킷런 라이브러리는 일반적으로 누락된 데이터를처리하지 못함
#누락된 값이 있는지 살펴본다
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [73]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [74]:
#학습 데이터셋의 중간값을 채워 넣는것으로 처리
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [75]:
#모델을 명세하자. IsFemale로 Sex열을 인코딩
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [76]:
#몇가지 모델변수를 선언하고 넘파이 배열을 생성
predictors = ['Pclass','IsFemale','Age']
X_train = train[predictors].to_numpy()
X_test = test[predictors].to_numpy()
y_train = train['Survived'].to_numpy()
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [77]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

In [79]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [81]:
model.fit(X_train,y_train)

LogisticRegression()

In [82]:
#테스트 데이터셋으로 예측
y_predict = model.predict(X_test)

In [83]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

- 오버피팅 되는것을 피할 수 있도록 교차검증 같은 기법을 활용
- MSE의 경우도 모델 정확도 점수에 기반에 모델 매개변수에 대한 그리드 검색을 수행
- LogisticResgressionCV 클래스
  - 모델 정규화 매개변수 C에 대한 그리드 검색을 얼마나 정밀하게 수행할 것인지 나타내는 매개변수와 함께 사용

In [85]:
from sklearn.model_selection import cross_val_score
model = LogisticRegression(C=10)
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])